In [ ]:
import pandas as pd
import numpy as np
from anomalous_edited import *
from function import *

df = pd.read_csv('forestfires.csv')

#Test to see which one is better
#df_normalized = normalization_zscore(df)
##df_normalized = normalization_range(df)

#number of data points
nn = df.shape[0]
#number of features
mm = df.shape[1]

print("Number of data points: ", nn)
print("Number of features: ", mm)

Number of data points:  517
Number of features:  11


In [ ]:
# Calculate global statistics
me = [np.mean(df.iloc[:, j]) for j in range(mm)]
mmax = [np.max(df.iloc[:, j]) for j in range(mm)]
mmin = [np.min(df.iloc[:, j]) for j in range(mm)]
ranges = []
normalization = 0  # set to 1 to ignore feature ranges, 0 otherwise
for j in range(mm):
    if normalization:
        ranges.append(1)
    else:
        rng = mmax[j] - mmin[j]
        if rng == 0:
            print("Variable num {} is constant!".format(j))
            rng = 1
        ranges.append(rng)

sy = np.divide((df - me), ranges)
sY = np.array(sy)
d = np.sum(sY * sY)   # total data scatter of normalized data

print("Mean: ", me)
print("Max: ", mmax)
print("Min: ", mmin)

Mean:  [np.float64(4.669245647969052), np.float64(4.299806576402321), np.float64(90.64468085106384), np.float64(110.87234042553192), np.float64(547.9400386847195), np.float64(9.021663442940039), np.float64(18.88916827852998), np.float64(44.28820116054158), np.float64(4.017601547388782), np.float64(0.021663442940038684), np.float64(12.847292069632493)]
Max:  [np.int64(9), np.int64(9), np.float64(96.2), np.float64(291.3), np.float64(860.6), np.float64(56.1), np.float64(33.3), np.int64(100), np.float64(9.4), np.float64(6.4), np.float64(1090.84)]
Min:  [np.int64(1), np.int64(2), np.float64(18.7), np.float64(1.1), np.float64(7.9), np.float64(0.0), np.float64(2.2), np.int64(15), np.float64(0.4), np.float64(0.0), np.float64(0.0)]


In [ ]:
x = df.values.astype(np.float32)
remains = list(range(nn))
threshold = 25
numberC = 0
ancl = []


while len(remains) > 0:

    # Step 1: Compute distance to the global mean
    distance = dist(x, remains, ranges, me)

    # Step 2: Choose the most distant point as centroid
    ind = np.argmax(distance)
    index = remains[ind]
    centroid = x[index, :]

    # Step 3: Run the anomalous pattern algorithm
    cluster, centroid = anomalousPattern(x, remains, ranges, centroid, me)

    if len(cluster) == 0:
        break

    # Step 4: Compute standardized centroid and contribution
    censtand = np.divide((np.asarray(centroid) - me), np.asarray(ranges))
    dD = np.sum(censtand * censtand.T * len(cluster) * 100 / d)

    # Step 5: Store the result
    ancl.append({
        'cluster': cluster,
        'centroid': censtand,
        'dD': dD
    })

    # Step 6: Update remains
    remains = np.setdiff1d(remains, cluster)
    numberC += 1


⏳ Cluster 1 | Points remaining: 517
✅ Cluster 1 done. Size: 102, Contribution: 22.5%
⏳ Cluster 2 | Points remaining: 415
✅ Cluster 2 done. Size: 1, Contribution: 0.68%
⏳ Cluster 3 | Points remaining: 414
✅ Cluster 3 done. Size: 154, Contribution: 11.22%
⏳ Cluster 4 | Points remaining: 260
✅ Cluster 4 done. Size: 1, Contribution: 0.5%
⏳ Cluster 5 | Points remaining: 259
✅ Cluster 5 done. Size: 153, Contribution: 12.36%
⏳ Cluster 6 | Points remaining: 106
✅ Cluster 6 done. Size: 34, Contribution: 2.64%
⏳ Cluster 7 | Points remaining: 72
✅ Cluster 7 done. Size: 1, Contribution: 0.3%
⏳ Cluster 8 | Points remaining: 71
✅ Cluster 8 done. Size: 10, Contribution: 0.61%
⏳ Cluster 9 | Points remaining: 61
✅ Cluster 9 done. Size: 44, Contribution: 1.04%
⏳ Cluster 10 | Points remaining: 17
✅ Cluster 10 done. Size: 3, Contribution: 0.2%
⏳ Cluster 11 | Points remaining: 14
✅ Cluster 11 done. Size: 1, Contribution: 0.11%
⏳ Cluster 12 | Points remaining: 13
✅ Cluster 12 done. Size: 1, Contribution: 0.

In [ ]:
# Filter clusters by threshold size
filtered_ancl = [ac for ac in ancl if len(ac['cluster']) >= threshold]

if len(filtered_ancl) == 0:
    print('Too great a threshold!!!')
else:
    # For demonstration, print out the clusters and their standardized centroids
    for i, ac in enumerate(filtered_ancl):
        print(f"Cluster {i + 1}:")
        print("  Size:", len(ac['cluster']))
        print("  Centroid (standardized):", np.round(ac['centroid'], 3))
        print("  Cluster contribution (%):", np.round(ac['dD'], 3))

Cluster 1:
  Size: 102
  Centroid (standardized): [ 0.032  0.026 -0.057 -0.287 -0.523 -0.061 -0.236  0.024  0.088 -0.003
 -0.006]
  Cluster contribution (%): 22.504
Cluster 2:
  Size: 154
  Centroid (standardized): [ 0.308  0.103  0.016  0.108  0.134  0.014  0.07  -0.013 -0.036 -0.001
  0.009]
  Cluster contribution (%): 11.224
Cluster 3:
  Size: 153
  Centroid (standardized): [-0.319 -0.1    0.019  0.093  0.151  0.018  0.099 -0.048 -0.033 -0.003
 -0.002]
  Cluster contribution (%): 12.363
Cluster 4:
  Size: 34
  Centroid (standardized): [-0.014 -0.043 -0.02   0.022  0.151 -0.003 -0.143  0.303  0.113  0.005
 -0.007]
  Cluster contribution (%): 2.638
Cluster 5:
  Size: 44
  Centroid (standardized): [-0.01  -0.053  0.015 -0.04   0.177 -0.     0.019 -0.078 -0.061 -0.003
 -0.004]
  Cluster contribution (%): 1.045
